In [1]:
get_ipython().magic('matplotlib inline')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook')

data=pd.read_excel('caschool.xlsx.xls')

state=440232650+470353886+470352982 # sum of the student IDs for the members of the group

train = data.sample(frac=0.8, random_state=state) # For tasks 1-5 
test = data[data.index.isin(train.index)==False].copy() # Only for prediction (task 6)

train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [2]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

C:\Users\sarah\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


I've just done an example model here, the code will be the same for all models 

In [3]:
# enter the formula and create the OLS for each of the 5 best models:
formula='testscr ~ str + avginc + el_pct + expn_stu + comp_stu'
ols1 = smf.ols(formula=formula, data=train).fit()
resid1 = ols1.resid
fitted1 = ols1.fittedvalues
ols1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                testscr   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     162.5
Date:                Mon, 30 Oct 2017   Prob (F-statistic):           9.93e-87
Time:                        20:44:09   Log-Likelihood:                -1256.7
No. Observations:                 336   AIC:                             2525.
Df Residuals:                     330   BIC:                             2548.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    640.2120     12.543     51.041      0.000     615.538     664.886
str           -0.0034      0.392     -0.009      0.993      -0.775       0.768
avginc         1.5005      0.088     17.106      0.000       1.328       1.673
el_pct        -0.4803      0.034    -14.186      0.000      -0.547      -0.414
expn_stu      -0.0008      0.001     -0.701      0.484      -0.003       0.002
comp_stu      21.9546      9.473      2.318      0.021       3.320      40.589
==============================================================================
Omnibus:                        7.234   Durbin-Watson:                   1.903
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                8.412
Skew:                          -0.225   Prob(JB):                       0.0149
Kurtosis:                       3.631   Cond. No.                     1.20e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
# Predict the test data: input requires all of the variables in the chosen model
yp_ff=ols1.predict({'str': test['str'], 'avginc': test['avginc'], 'el_pct': test['el_pct'], 'expn_stu':test['expn_stu'], 'comp_stu': test['comp_stu']})

In [6]:
#RMSFE, MAFE
(sum((test['testscr']-yp_ff)**2)/len(yp_ff))**0.5, sum(abs(test['testscr']-yp_ff))/len(yp_ff)

(10.516902603123329, 8.7051769667957757)

In [7]:
# to calculate forecast R-squared
formula='testscr ~ yp_ff'
ols1_t = smf.ols(formula=formula, data=test).fit()
print(ols1_t.summary())

                            OLS Regression Results                            
Dep. Variable:                testscr   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     195.9
Date:                Mon, 30 Oct 2017   Prob (F-statistic):           1.92e-23
Time:                        20:51:44   Log-Likelihood:                -316.44
No. Observations:                  84   AIC:                             636.9
Df Residuals:                      82   BIC:                             641.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     32.8663     44.489      0.739      0.4